1. 筛选出不合格的cDCs, pDCs
2. 分别计算他们对应的log2FC, p_value, gene_exp, ave_exp
3. 替换到现在的表格

In [103]:
fn_int = '/home/yike/phd/dengue/data/interaction_source_file/interactions_DB.tsv'
interactions = pd.read_csv(fn_int, sep=',')[['gene_name_a', 'gene_name_b']]
genes = np.unique(interactions)
#genes = [i for i in genes if i in adata.var_names]
genes = [i for i in genes if i not in ['CCL3L3', 'CCL4L1', 'CCN6', 'KIR3DS1', 'YARS1']]

Load interaction


In [132]:
from log2_FC_functions import log2_FC_all

log2_fc_ave = log2_FC_all(genes, adata_cDCs, 'S_dengue', 'dengue', 'child')[1]
log2_fc_ave['cell_type']=['cDCs']*log2_fc_all.shape[0]

log2_fc_ave['gene'] = log2_fc_ave.index.values
log2_fc_ave = log2_fc_ave.set_index('cell_type')
log2_fc_ave.to_csv('/home/yike/phd/dengue/data/excels/log2_fc/S_dengue_vs_dengue/log2_fc_ave_kids_cDCs.tsv')

/usr/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [125]:
adatag = adata_cDCs[:, genes]

from numpy import * # 调用numpy所有函数
adatag_children = adatag[adatag.obs['dataset'] == 'child']

IDs = list(adatag_children.obs['ID'].astype('category').cat.categories)

#cell_types = ['B_cells', 'Monocytes', 'NK_cells', 'Plasmablasts', 'T_cells', 'cDCs', 'pDCs']
conditions = ['S_dengue', 'dengue', 'DWS', 'Healthy']
from collections import defaultdict
adatag_ch_ct_cd_ID = {}
for condition in conditions:
    for ID in IDs:
        adatag_ch_ct_cd_ID[(condition, ID)] = adatag_children[adatag_children.obs['Condition'] == condition][adatag_children[adatag_children.obs['Condition'] == condition].obs['ID'] == ID]

gene_exp_ave = {}
ave_exp_ave = {}
for condition in conditions:
    exp_fra = []
    ave_exp = []
    for ID in list(adatag_children[adatag_children.obs['Condition'] == condition].obs['ID'].astype('category').cat.categories):
        exp_fra.append((adatag_ch_ct_cd_ID[(condition, ID)].X > 0).toarray().mean(axis=0))
        ave_exp.append(adatag_ch_ct_cd_ID[(condition, ID)].X.toarray().mean(axis=0))
        
    exp_fra = np.array(exp_fra).mean(axis=0)
    ave_exp = np.array(ave_exp).mean(axis=0)
    
    gene_exp_ave[condition] = pd.DataFrame(exp_fra, index=adatag_children[adatag_children.obs['Condition'] == condition].var.index, columns=['gene_expre'])
    ave_exp_ave[condition] = pd.DataFrame(ave_exp, index=adatag_children[adatag_children.obs['Condition'] == condition].var.index, columns=['ave_exp'])
    
exp_fra_ave = pd.DataFrame([])
ave_exp = pd.DataFrame([])
for key in gene_exp_ave.keys():
    gene_exp_ave[key]['cell_type'] = ['cDCs']*(gene_exp_ave[key].shape[0])
    gene_exp_ave[key]['condition'] = [key]*(gene_exp_ave[key].shape[0])
    exp_fra_ave = pd.concat([exp_fra_ave, gene_exp_ave[key]])
    exp_fra_ave['gene'] = exp_fra_ave.index
    exp_fra_ave.to_csv('/home/yike/phd/dengue/data/excels/log2_fc/S_dengue_vs_dengue/exp_fra_ave_cDCs.tsv')
    
    ave_exp_ave[key]['cell_type'] = ['cDCs']*(ave_exp_ave[key].shape[0])
    ave_exp_ave[key]['condition'] = [key]*(ave_exp_ave[key].shape[0])
    ave_exp = pd.concat([ave_exp, ave_exp_ave[key]])
    ave_exp['gene'] = ave_exp.index
    ave_exp.to_csv('/home/yike/phd/dengue/data/excels/log2_fc/S_dengue_vs_dengue/ave_exp_ave_cDCs.tsv')

/usr/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [216]:
log2_fc_cDCs = pd.read_csv('/home/yike/phd/dengue/data/excels/log2_fc/S_dengue_vs_dengue/log2_fc_ave_kids_cDCs.tsv', index_col=['cell_type', 'gene'])
gene_exp_cDCs = pd.read_csv('/home/yike/phd/dengue/data/excels/log2_fc/S_dengue_vs_dengue/exp_fra_ave_cDCs.tsv', index_col=['cell_type', 'gene', 'condition'])
ave_exp_cDCs = pd.read_csv('/home/yike/phd/dengue/data/excels/log2_fc/S_dengue_vs_dengue/ave_exp_ave_cDCs.tsv', index_col=['cell_type', 'gene', 'condition'])

In [217]:
idx_SD = [(i[0], i[1], 'S_dengue') for i in log2_fc_cDCs.index]
idx_D = [(i[0], i[1], 'dengue') for i in log2_fc_cDCs.index]

In [218]:
gene_exp_cDCs_SD = gene_exp_cDCs.loc[idx_SD]
gene_exp_cDCs_D = gene_exp_cDCs.loc[idx_D]
ave_exp_cDCs_SD = ave_exp_cDCs.loc[idx_SD]
ave_exp_cDCs_D = ave_exp_cDCs.loc[idx_D]

In [219]:
#SD_exp_frac	D_exp_frac	SD_ave_exp	D_ave_exp
log2_fc_cDCs['SD_exp_frac'] = gene_exp_cDCs_SD['gene_expre'].tolist()
log2_fc_cDCs['D_exp_frac'] = gene_exp_cDCs_D['gene_expre'].tolist()
log2_fc_cDCs['SD_ave_exp'] = ave_exp_cDCs_SD['ave_exp'].tolist()
log2_fc_cDCs['D_ave_exp'] = ave_exp_cDCs_D['ave_exp'].tolist()

In [211]:
log2_fc = pd.read_csv('/home/yike/phd/dengue/data/excels/log2_fc/S_dengue_vs_dengue/log2_fc_ave_kids.tsv', index_col=['cell_type', 'gene'])
gene_exp = pd.read_csv('/home/yike/phd/dengue/data/excels/log2_fc/S_dengue_vs_dengue/exp_fra_ave.tsv', index_col=['cell_type', 'gene', 'condition'])
ave_exp = pd.read_csv('/home/yike/phd/dengue/data/excels/log2_fc/S_dengue_vs_dengue/ave_exp_ave.tsv', index_col=['cell_type', 'gene', 'condition'])

In [214]:
log2_fc_new = log2_fc.loc[['B_cells', 'Monocytes', 'NK_cells', 'Plasmablasts', 'T_cells', 'pDCs']]
log2_fc = pd.concat([log2_fc_new, log2_fc_cDCs])
log2_fc.to_csv('/home/yike/phd/dengue/data/excels/log2_fc/S_dengue_vs_dengue/log2_fc_ave_kids.tsv')

In [230]:
gene_exp_new = gene_exp.loc[['B_cells', 'Monocytes', 'NK_cells', 'Plasmablasts', 'T_cells', 'pDCs']]
gene_exp = pd.concat([gene_exp_new, gene_exp_cDCs])
gene_exp.to_csv('/home/yike/phd/dengue/data/excels/log2_fc/S_dengue_vs_dengue/exp_fra_ave.tsv')

In [233]:
ave_exp_new = ave_exp.loc[['B_cells', 'Monocytes', 'NK_cells', 'Plasmablasts', 'T_cells', 'pDCs']]
ave_exp = pd.concat([ave_exp_new, ave_exp_cDCs])
ave_exp.to_csv('/home/yike/phd/dengue/data/excels/log2_fc/S_dengue_vs_dengue/ave_exp_ave.tsv')